In [ ]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from dash.dependencies import Input, Output
from sklearn import datasets
from sklearn.cluster import KMeans
from pages import demo1_datatable, demo2_predmain

In [ ]:
BS = "https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/flatly/bootstrap.min.css"
app = dash.Dash(external_stylesheets=[BS])

In [ ]:
# list seperate sub 
subpages = [
    ("/demo-datatable", demo1_datatable),
    ("/demo-pm", demo2_predmain),
]

In [ ]:

df = pd.read_csv('screwdriverstatisticsII.csv', sep='\s+', delimiter=';', header=0, parse_dates=True, squeeze=True, decimal=",")
df.head(100)
df.columns

In [ ]:
## Creating very simple graph here 
fig = px.scatter(
    df,
    x='RA',
    y='T1',
    color='UnitPartNo',
    marginal_x="violin",
    marginal_y="violin",
    template="simple_white"
    #facet_col='Type',
    #template='presentation',
    )
#fig.update_traces(mode='lines+markers')

In [ ]:
fig2 = px.scatter(
    df,
    x='RA',
    y='T2',
    color='UnitPartNo',
    marginal_x="violin",
    marginal_y="violin",
    template="simple_white"
    #facet_col='Type',
    #template='presentation',
    )
#fig.update_traces(mode='lines+markers')

In [ ]:
df_small = df[['UnitPartNo', 'UnitSnNo', 'Seq', 'PN', 'PT', 'RC', 'ScrewdriverId']].copy()
df_small

fig3 = px.parallel_coordinates(df, color="PN", labels={"UnitPartNo": "Product Types",
                  "Seq": "Sequence - Screw Number", "PN": "Program number",
                  "ScrewdriverId": "Screwdriver ID", "RC": "Result Code", },
                    color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=2)
fig3.show()

In [ ]:
fig3 = px.parallel_coordinates(df_small, color="PN", color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=2)
fig3.show()


In [ ]:

fig4 = px.parallel_categories(df, color="Seq", color_continuous_scale=px.colors.sequential.Inferno)
fig4.show()

In [ ]:
fig5 = px.line(df, x = 'TimeStamp', y = 'T1', color = 'UnitPartNo')
fig5.show()

In [ ]:
fig6 = px.line(df, x = 'TimeStamp', y = 'ET', color = 'UnitPartNo')
fig6.show()

In [ ]:
fig7 = px.scatter(df, x = 'TimeStamp', y = 'PN', color = 'UnitPartNo')
fig7.show()

In [ ]:
# Builidng the data table 
the_datatable = dash_table.DataTable(id='table1', 
columns=[{"name": i, "id": i } for i in df.columns], 
data=df.to_dict('records'), 
)

In [ ]:
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("X variable"),
                dcc.Dropdown(
                    id="x-variable",
                    options=[
                        {"label": col, "value": col} for col in df.columns
                    ],
                    value="RA",
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Y variable"),
                dcc.Dropdown(
                    id="y-variable",
                    options=[
                        {"label": col, "value": col} for col in df.columns
                    ],
                    value="T1",
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Cluster count"),
                dbc.Input(id="cluster-count", type="number", value=3),
            ]
        ),
    ],
    body=True,
)


In [ ]:
opts = [ dict(label=t, value=t)
         for t in df['UnitPartNo'].unique() ]

In [ ]:
nav = dbc.Nav(
    [
        dbc.NavItem(dbc.NavLink("Visual Graphs", active=True, href="1")),
        dbc.NavItem(dbc.NavLink("Entire Table", href="2",)),
        dbc.NavItem(dbc.NavLink("Predictive Maintenance", href="3")), 
        dbc.NavItem(dbc.NavLink("GitHub Repo", href="https://github.com/")),
        #dbc.NavItem(dbc.NavLink("Disabled", disabled=True, href="4")),
    ],
    pills=True,
    horizontal="center", 
)
@app.callback(
    Output("button-clicks", "children"), [Input("button-link", "n_clicks")]
)
def show_clicks(n):
    return "Button clicked {} times".format(n)

In [ ]:
## App Layout

In [ ]:
app.layout = dbc.Container(
    [
        nav,
        html.H1("Interactive Screwdriver Statistics"),
        html.Hr(),
        dcc.Markdown('''
                ### Clustering Torque Values against Result Angle
                
                Visualizes outliers both for torque values under/over- performing as well as Result Angles overshooting.
        '''),
        dbc.Row(
            [
                dcc.Graph(figure=fig), 
                dcc.Graph(figure=fig2),
                dcc.Dropdown(id='select-type',
                            options=opts, 
                            multi=True, 
                            value='Total'), 
               # dcc.Graph(id='graph')
            ], 
            align="center", 
        ),
        dcc.Markdown('''
                ### Screwdriver and Program Usage
        '''),
         dbc.Row(
            [
                dcc.Graph(figure=fig3),
                dcc.Graph(figure=fig7), 
                # the_datatable, # havent figured out yet how to limit table to top 10 or something
                #dcc.Graph(figure=fig4),
               
            ], 
            align="center", 
        ),
        dcc.Markdown('''
                ### Timeseries Data
        '''),
         dbc.Row(
            [
                dcc.Graph(figure=fig5),
                dcc.Graph(figure=fig6), 
                # the_datatable, # havent figured out yet how to limit table to top 10 or something
                #dcc.Graph(figure=fig4),
               
            ], 
            align="center", 
        ),
        dcc.Markdown('''
                ### Custom Clustering with all features from the dataset
                
                Select two values in the dropdown menu to create a custom interactive scatterplot. (Not all values are suited for clustering)
        '''),
        dbc.Row(
            [
                dbc.Col(controls, md=4),
                dbc.Col(dcc.Graph(id="cluster-graph"), md=8),
            ],
            align="center",
        ),

    ],
    fluid=True,
)

In [ ]:
# Subpages callbacks
# Set callbacks for each page
for (_, page) in subpages:
    page.set_callbacks(app)

def display_page(pathname):
    """Render the newly selected page when the URL changes"""

    if pathname == "/":
        return main_layout

    page = dict(subpages)[pathname]

    return html.Div(
        [
            # For subpages, add a few fixed elements at the top of the page
            dcc.Link("< Back to main page", href="/"),
            html.H1(page.title),
            html.P(page.description),
            # Then, the actual subpage content
            page.layout,
        ]
    )

In [ ]:
##Callback 

In [ ]:
@app.callback(
    Output("cluster-graph", "figure"),
    [
        Input("x-variable", "value"),
        Input("y-variable", "value"),
        Input("cluster-count", "value"),
    ],
    
)
##Functions
def make_graph(x, y, n_clusters):
    # minimal input validation, make sure there's at least one cluster
    km = KMeans(n_clusters=max(n_clusters, 1))
    df = df.loc[:, [x, y]]
    km.fit(df.values)
    df["cluster"] = km.labels_

    centers = km.cluster_centers_

    data = [
        go.Scatter(
            x=df.loc[df.cluster == c, x],
            y=df.loc[df.cluster == c, y],
            mode="markers",
            marker={"size": 8},
            name="Cluster {}".format(c),
        )
        for c in range(n_clusters)
    ]

    data.append(
        go.Scatter(
            x=centers[:, 0],
            y=centers[:, 1],
            mode="markers",
            marker={"color": "#000", "size": 12, "symbol": "diamond"},
            name="Cluster centers",
        )
    )

    layout = {"xaxis": {"title": x}, "yaxis": {"title": y}}

    return go.Figure(data=data, layout=layout)

def make_figure(select_type):
    select_type = select_type
    #if isinstance(select_type, list)
    # else [select_type]
    fig = px.line(
        df.loc[df['UnitPatNo'].isin(select_type)],
        x='Year',
        y='Homicides',
        color='Type',
        line_dash='Type',
        template='presentation',
        category_orders={'Type': ['Total']},
        )
    return fig.update_traces(mode='lines+markers')


# make sure that x and y values can't be the same variable
def filter_options(v):
    """Disable option v"""
    return [
        {"label": col, "value": col, "disabled": col == v}
        for col in df.columns
    ]

# functionality is the same for both dropdowns, so we reuse filter_options
app.callback(Output("x-variable", "options"), [Input("y-variable", "value")])(filter_options)

app.callback(
    Output("y-variable", "options"),
    [Input("x-variable", "value")
    ])(filter_options
)

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=False, port=8000)